# Seylan

In [1]:
##Monte carlo Simulation for decay factor = 0 and holding period =1

In [2]:
##Importing necessary libraries
import math
import pandas as pd
import numpy as np
import scipy
import random
import math ##Importing Math function to use floor function.(Floor function rounds up the value to the nearest integer)
from scipy import stats
from csv import reader
random.seed(120)

In [3]:
path = "/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Equity/Data/rates_instrument.csv"
path1 = "/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Equity/Data/rates_interest.csv"
path2 = "/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Equity/Data/rates_exchange.csv"
path3 = "/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Equity/Data/transaction_equity.csv"
path4 = "/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Equity/Data/parameters_hp1.csv"


In [4]:
def getData(path, fileext = None):
    if fileext == ".csv":
        df = pd.read_csv(path)
    elif fileext == ".xlsx" or fileext == ".xls":
        df = pd.read_excel(path)
    else:
        df = pd.read_csv(path)
    return df

In [5]:
##Importing dataset rates_instrument
df= getData(path)


In [6]:
#Function to get a specific type of data from the particular column in data(Filtering)
def loadData(df, colname, types = None):
    
    res_df = df[df[colname] == types].reset_index(drop=True)
    return res_df
    

In [7]:
df_D = loadData(df,'instrument_type',types = 'D')
df_D
    

,date,instrument,instrument_type,value,value_stressed
0,2020-10-16,LKA36421B051,D,4.610000,4.610000
1,2020-10-15,LKA36421B051,D,4.610000,4.610000
2,2020-10-14,LKA36421B051,D,4.620000,4.620000
3,2020-10-13,LKA36421B051,D,4.620000,4.620000
4,2020-10-12,LKA36421B051,D,4.620000,4.620000
...,...,...,...,...,...
2003,2019-11-07,LKB01326B011,D,10.320000,10.320000
2004,2019-11-06,LKB01326B011,D,10.320000,10.320000
2005,2019-11-05,LKB01326B011,D,10.320000,10.320000
2006,2019-11-04,LKB01326B011,D,10.320000,10.320000


In [8]:
def sim_returns(df, colname = None, value = None):
    simple_returns = pd.DataFrame()

    for a, g in df.groupby(colname):

        g = g.reset_index(drop = True)
        g['simple_returns'] = g[value]-g[value].shift(-1)
        g = g.iloc[:-1 , :]
        simple_returns = simple_returns.append(g)
        
    
    return simple_returns

In [9]:
df_D = sim_returns(df_D, colname="instrument", value = "value")


/tmp/ipykernel_175/947502788.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_175/947502788.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_175/947502788.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_175/947502788.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_175/947502788.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

In [10]:
df_D.iloc[499]

date                 2019-11-04
instrument         LKA36421I031
instrument_type               D
value                      4.87
value_stressed             4.87
simple_returns              0.0
Name: 249, dtype: object

In [13]:
##Generating Uniform random numbers function
random.seed(120)
def generateRandoms(a,b,n):
    randoms = []
    for x in range(0,n):
        randoms.append(random.uniform(a,b))
    return randoms

In [14]:
##Generating Uniform random numbers function for D set
random_numbers_list = generateRandoms(0.000000001,0.000000010,2008) 
# a = 0.000000001
# b = 0.000000010
# n = 2008)

In [15]:
df_D['returns_adjustment_factor'] = pd.Series(generateRandoms(0.000000001,0.000000010,2008))  ##pd.series creates a series from an array
df_D.head()

,date,instrument,instrument_type,value,value_stressed,simple_returns,returns_adjustment_factor
0,2020-10-16,LKA36421B051,D,4.61,4.61,0.00,6.620800e-09
1,2020-10-15,LKA36421B051,D,4.61,4.61,-0.01,6.944767e-09
2,2020-10-14,LKA36421B051,D,4.62,4.62,0.00,6.887026e-09
3,2020-10-13,LKA36421B051,D,4.62,4.62,0.00,8.685328e-09
4,2020-10-12,LKA36421B051,D,4.62,4.62,0.01,8.319420e-09


In [16]:
def simulations(df, col1,col2):
    df["simulated_returns"] = df[col1]+ df[col2]
    return df

In [17]:
df_D = simulations(df_D,'simple_returns','returns_adjustment_factor')
df_D.head()

,date,instrument,instrument_type,value,value_stressed,simple_returns,returns_adjustment_factor,simulated_returns
0,2020-10-16,LKA36421B051,D,4.61,4.61,0.00,6.620800e-09,6.620800e-09
1,2020-10-15,LKA36421B051,D,4.61,4.61,-0.01,6.944767e-09,-9.999993e-03
2,2020-10-14,LKA36421B051,D,4.62,4.62,0.00,6.887026e-09,6.887026e-09
3,2020-10-13,LKA36421B051,D,4.62,4.62,0.00,8.685328e-09,8.685328e-09
4,2020-10-12,LKA36421B051,D,4.62,4.62,0.01,8.319420e-09,1.000001e-02


In [18]:
###Filtering according to Instrument
df_res_D = df_D.filter(['instrument', 'simulated_returns']) ##To extract particular columns

In [19]:
df_res_D = df_res_D.set_index('instrument')
df_res_D

,simulated_returns
instrument,
LKA36421B051,6.620800e-09
LKA36421B051,-9.999993e-03
LKA36421B051,6.887026e-09
LKA36421B051,8.685328e-09
LKA36421B051,1.000001e-02
...,...
LKB01326B011,4.292624e-09
LKB01326B011,7.419903e-09
LKB01326B011,3.080865e-09


In [20]:
##Segregating simulations according to the instrument
new_df_D = (df_res_D.assign(labels = df_res_D.groupby(level = 0).cumcount())
            .groupby([df_res_D.index,'labels']).first()
            .unstack('labels')
            .sort_index(axis =1,level = 1)
            .droplevel(1,axis = 1))

In [21]:
new_df_D

,simulated_returns,simulated_returns,simulated_returns,simulated_returns,simulated_returns,simulated_returns,simulated_returns,simulated_returns,simulated_returns,simulated_returns,...,simulated_returns,simulated_returns,simulated_returns,simulated_returns,simulated_returns,simulated_returns,simulated_returns,simulated_returns,simulated_returns,simulated_returns
instrument,,,,,,,,,,,,,,,,,,,,,
LKA36421B051,6.620800e-09,-9.999993e-03,6.887026e-09,8.685328e-09,0.010000,8.161819e-09,-9.999995e-03,2.000000e-02,5.845344e-09,2.000001e-02,...,5.330123e-09,1.309363e-09,1.903723e-09,7.720811e-09,6.941902e-09,4.292624e-09,7.419903e-09,3.080865e-09,1.637769e-09,4.055219e-09
LKA36421I031,6.620800e-09,6.944767e-09,6.887026e-09,8.685328e-09,-0.010000,8.161819e-09,4.645956e-09,2.000000e-02,5.845344e-09,7.368540e-09,...,5.330123e-09,1.309363e-09,1.903723e-09,7.720811e-09,6.941902e-09,4.292624e-09,7.419903e-09,3.080865e-09,1.637769e-09,4.055219e-09
LKB00523C152,6.620800e-09,-2.916666e-02,-2.083333e-02,-4.833333e-02,-0.010000,-8.333325e-03,-7.166666e-02,1.833334e-02,-4.999994e-03,-3.666666e-02,...,5.330123e-09,2.097222e-02,1.444445e-02,7.720811e-09,6.941902e-09,4.292624e-09,5.555562e-03,3.080865e-09,1.637769e-09,1.349207e-02
LKB00524C150,6.620800e-09,-3.773809e-02,-4.416666e-02,-4.166666e-02,-0.016667,-2.083332e-02,-3.416666e-02,1.509622e-09,8.333340e-03,-8.333333e-02,...,5.330123e-09,1.250000e-02,2.500002e-03,7.720811e-09,6.941902e-09,4.292624e-09,3.750007e-03,3.080865e-09,6.250002e-03,1.875000e-02
LKB00524I157,-1.999999e-02,-2.055555e-02,-3.444444e-02,-6.111110e-02,-0.025167,-3.061110e-02,-6.422222e-02,-2.222220e-03,-2.777772e-03,-7.111110e-02,...,1.000005e-03,1.400000e-02,-1.999998e-03,7.720811e-09,6.941902e-09,4.292624e-09,1.700001e-02,-6.999997e-03,5.000002e-03,2.345455e-02
LKB00623E156,6.620800e-09,-4.999999e-02,-4.166659e-03,-4.999999e-02,-0.012500,-8.333326e-03,-8.333333e-02,1.666667e-02,-8.333328e-03,-4.333333e-02,...,5.330123e-09,1.000000e-02,3.000000e-02,7.720811e-09,6.941902e-09,4.292624e-09,7.419903e-09,-3.999997e-03,4.000002e-03,1.000000e-02
LKB01026H014,-3.499999e-02,-9.999993e-03,-9.999993e-03,-4.999999e-02,-0.010000,8.161819e-09,-9.500000e-02,-2.960000e-01,1.600001e-02,2.250000e-01,...,1.696430e-02,2.142857e-02,-7.142858e-03,7.720811e-09,6.941902e-09,4.292624e-09,7.142867e-03,-7.142857e-03,1.637769e-09,1.875000e-02
LKB01326B011,-2.055555e-02,-3.833333e-02,-2.722222e-02,-3.944444e-02,-0.025556,-3.111110e-02,-1.033333e-01,7.500002e-03,-1.874994e-03,-9.249999e-02,...,5.330123e-09,1.309363e-09,1.903723e-09,7.720811e-09,6.941902e-09,4.292624e-09,7.419903e-09,3.080865e-09,1.637769e-09,1.666667e-02


In [22]:
new_df_D_T = new_df_D.transpose() ##Transposing rows to columns
new_df_D_T

instrument,LKA36421B051,LKA36421I031,LKB00523C152,LKB00524C150,LKB00524I157,LKB00623E156,LKB01026H014,LKB01326B011
simulated_returns,6.620800e-09,6.620800e-09,6.620800e-09,6.620800e-09,-1.999999e-02,6.620800e-09,-3.499999e-02,-2.055555e-02
simulated_returns,-9.999993e-03,6.944767e-09,-2.916666e-02,-3.773809e-02,-2.055555e-02,-4.999999e-02,-9.999993e-03,-3.833333e-02
simulated_returns,6.887026e-09,6.887026e-09,-2.083333e-02,-4.416666e-02,-3.444444e-02,-4.166659e-03,-9.999993e-03,-2.722222e-02
simulated_returns,8.685328e-09,8.685328e-09,-4.833333e-02,-4.166666e-02,-6.111110e-02,-4.999999e-02,-4.999999e-02,-3.944444e-02
simulated_returns,1.000001e-02,-9.999992e-03,-9.999992e-03,-1.666666e-02,-2.516666e-02,-1.249999e-02,-9.999992e-03,-2.555555e-02
...,...,...,...,...,...,...,...,...
simulated_returns,4.292624e-09,4.292624e-09,4.292624e-09,4.292624e-09,4.292624e-09,4.292624e-09,4.292624e-09,4.292624e-09
simulated_returns,7.419903e-09,7.419903e-09,5.555562e-03,3.750007e-03,1.700001e-02,7.419903e-09,7.142867e-03,7.419903e-09
simulated_returns,3.080865e-09,3.080865e-09,3.080865e-09,3.080865e-09,-6.999997e-03,-3.999997e-03,-7.142857e-03,3.080865e-09
simulated_returns,1.637769e-09,1.637769e-09,1.637769e-09,6.250002e-03,5.000002e-03,4.000002e-03,1.637769e-09,1.637769e-09


In [23]:
new_df_D_T = new_df_D_T.reset_index(drop = True) ##dropping the instrument column

In [24]:
#new_df_D_T = pd.DataFrame(new_df_D_T)

# E

In [25]:
##Extracting data with Instrument type 'E' from Rate_Instrument dataset
df_E = loadData(df,'instrument_type',types = 'E')
   

In [26]:
def log_returns(df, colname = None, value = None):
    log_returns= pd.DataFrame()

    for a, g in df.groupby(colname):

        g = g.reset_index(drop = True)
        const_number = g[value][0]
        numerator = g[value]
        denominator = g[value].shift(-1)
#         print(np.log(numerator/denominator))
        g['log_returns']=np.log(numerator/denominator)
        g = g.iloc[:-1 , :]
        log_returns = log_returns.append(g)
        
    
    return log_returns

In [27]:
df_E = log_returns(df_E, colname = 'instrument', value = 'value')

/tmp/ipykernel_175/959808997.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_returns = log_returns.append(g)
/tmp/ipykernel_175/959808997.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_returns = log_returns.append(g)
/tmp/ipykernel_175/959808997.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_returns = log_returns.append(g)
/tmp/ipykernel_175/959808997.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_returns = log_returns.append(g)
/tmp/ipykernel_175/959808997.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  l

In [26]:
df_E.iloc[250]

date               2020-10-16
instrument          JETWN0000
instrument_type             E
value                     9.1
value_stressed            9.1
log_returns           0.01105
Name: 0, dtype: object

In [28]:
##Generating random numbers
# a = 0.0000001
# b = 0.0000010
# n = 1506

df_E['returns_adjustment_factor'] = pd.Series(generateRandoms(0.0000001,0.0000010,1506))
df_E.head()

,date,instrument,instrument_type,value,value_stressed,log_returns,returns_adjustment_factor
0,2020-10-16,GLASN0000,E,7.1,7.1,-0.119347,9.171972e-07
1,2020-10-15,GLASN0000,E,8.0,8.0,-0.095310,1.812227e-07
2,2020-10-14,GLASN0000,E,8.8,8.8,0.228842,6.363912e-07
3,2020-10-13,GLASN0000,E,7.0,7.0,0.336472,2.516031e-07
4,2020-10-12,GLASN0000,E,5.0,5.0,0.174353,9.081317e-07


In [29]:
df_E = simulations(df_E,'log_returns','returns_adjustment_factor')

In [30]:
###Filtering according to Instrument
df_res_E = df_E.filter(['instrument', 'simulated_returns']) ##To extract particular columns

In [31]:
df_res_E = df_res_E.set_index('instrument')

In [32]:
##Segregating simulations according to the instrument
new_df_E = (df_res_E.assign(labels = df_res_E.groupby(level = 0).cumcount())
            .groupby([df_res_E.index,'labels']).first()
            .unstack('labels')
            .sort_index(axis =1,level = 1)
            .droplevel(1,axis = 1))

In [33]:
new_df_E_T = new_df_E.transpose() ##Transposing rows to columns

In [34]:
new_df_E_T = new_df_E_T.reset_index(drop = True) ##dropping the instrument column

In [35]:
new_df_E_T

instrument,GLASN0000,JETWN0000,JKHN0000,KHLN0000,LIOCN0000,LLUBN0000
0,-1.193458e-01,1.105075e-02,-8.994646e-03,3.681489e-02,9.171972e-07,1.569631e-02
1,-9.531000e-02,-2.197873e-02,1.579592e-02,-1.242234e-02,2.874959e-02,3.448636e-02
2,2.288422e-01,2.197954e-02,1.759191e-02,1.242316e-02,1.680775e-02,1.770021e-02
3,3.364725e-01,2.516031e-07,-1.074429e-02,-3.681372e-02,-3.333617e-02,-9.478492e-03
4,1.743543e-01,1.117421e-02,-9.117696e-03,2.439236e-02,-4.089077e-03,8.289884e-03
...,...,...,...,...,...,...
245,6.630170e-07,6.630170e-07,6.630170e-07,6.630170e-07,6.630170e-07,6.630170e-07
246,9.101167e-07,9.101167e-07,-3.900265e-03,9.101167e-07,-6.005114e-03,-2.876716e-02
247,-2.409692e-02,6.358832e-07,6.497758e-04,-9.755539e-03,-5.969531e-03,-7.434343e-03
248,-2.353038e-02,1.129955e-07,-6.472402e-03,9.756288e-03,-1.183434e-02,-1.907603e-02


In [34]:
#new_df_E_T = pd.DataFrame(new_df_E_T)

# RATES INTEREST

In [36]:
##Importing dataset rates_interest
df1= getData(path1)
df1

,date,curve,tenor,rate,rate_stressed
0,2020-10-16,GOVSECBID,1w,4.503333,4.503333
1,2020-10-15,GOVSECBID,1w,4.503333,4.503333
2,2020-10-14,GOVSECBID,1w,4.513300,4.513300
3,2020-10-13,GOVSECBID,1w,4.513300,4.513300
4,2020-10-12,GOVSECBID,1w,4.513333,4.513333
...,...,...,...,...,...
3258,2019-11-07,GOVSECBID,1y,8.488889,8.488889
3259,2019-11-06,GOVSECBID,1y,8.466667,8.466667
3260,2019-11-05,GOVSECBID,1y,8.466667,8.466667
3261,2019-11-04,GOVSECBID,1y,8.466667,8.466667


In [37]:
df1["key1"] = df1["curve"] + '_' + df1["tenor"]

In [38]:
##Sorting data by'tenor' column
df1= df1.sort_values(by=['curve', 'tenor']).reset_index(drop=True)

In [39]:
df1 = sim_returns(df1, colname="key1", value = "rate")

/tmp/ipykernel_175/947502788.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_175/947502788.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_175/947502788.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_175/947502788.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_returns = simple_returns.append(g)
/tmp/ipykernel_175/947502788.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

In [40]:
df1['returns_adjustment_factor'] = pd.Series(generateRandoms(0.0000001,0.0000010,3763))

In [41]:
df1 = simulations(df1,'simple_returns','returns_adjustment_factor')

In [42]:
df_res1 = df1.filter(['key1', 'simulated_returns']) ##To extract particular columns

In [43]:
df_res1 = df_res1.set_index('key1')

In [44]:
##Segregating simulations according to the key1 generated
new_df1 = (df_res1.assign(labels = df_res1.groupby(level = 0).cumcount())
            .groupby([df_res1.index,'labels']).first()
            .unstack('labels')
            .sort_index(axis =1,level = 1)
            .droplevel(1,axis = 1))

In [45]:
new_df1_T = new_df1.transpose() ##Transposing rows to columns

In [46]:
new_df1_T = new_df1_T.reset_index(drop = True) ##dropping the instrument column

# EXCHANGE DATASET

In [47]:
##Importing dataset rates_interest
df2= getData(path2)
df2.head()


,date,currency_from,currency_to,rate,rate_stressed,currency_rate_increase,currency_rate_decrease
0,16-10-2020,LKR,LKR,1.0,1.0,1.0,1.0
1,15-10-2020,LKR,LKR,1.0,1.0,1.0,1.0
2,14-10-2020,LKR,LKR,1.0,1.0,1.0,1.0
3,13-10-2020,LKR,LKR,1.0,1.0,1.0,1.0
4,12-10-2020,LKR,LKR,1.0,1.0,1.0,1.0


In [48]:
df2["key2"] = df2["currency_from"] + '_' + df2["currency_to"]

In [49]:
df2 = log_returns(df2, colname = 'key2', value = 'rate')

/tmp/ipykernel_25134/959808997.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_returns = log_returns.append(g)
/tmp/ipykernel_25134/959808997.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_returns = log_returns.append(g)
/tmp/ipykernel_25134/959808997.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_returns = log_returns.append(g)


In [50]:
random_numbers_list = generateRandoms(0.0000001,0.0000010,753)


In [51]:
df2['returns_adjustment_factor'] = pd.Series(generateRandoms(0.0000001,0.0000010,753))


In [52]:
df2 = simulations(df2,'log_returns','returns_adjustment_factor')

In [53]:
df_res2 = df2.filter(['key2', 'simulated_returns']) ##To extract particular columns

In [54]:
df_res2 = df_res2.set_index('key2')

In [55]:
##Segregating simulations according to the key1 generated
new_df2 = (df_res2.assign(labels = df_res2.groupby(level = 0).cumcount())
            .groupby([df_res2.index,'labels']).first()
            .unstack('labels')
            .sort_index(axis =1,level = 1)
            .droplevel(1,axis = 1))


In [56]:
new_df2_T = new_df2.transpose() ##Transposing rows to columns

In [57]:
new_df2_T = new_df2_T.reset_index(drop = True) ##dropping the instrument column

In [ ]:
#From below cell all codes will wind up in a python script where it will call other 3 scripts for instrument types, exchange rates and rates interest

In [59]:
##Concatenating all the simulated columns in a single dataframe(Format:Interest+Exchange +D +E)
Combine=pd.concat([new_df1_T,new_df2_T,new_df_D_T,new_df_E_T],axis= 1)

In [60]:
Combine.columns ##To check the columns of the DataFrame

Index(['GOVSECBID_10m', 'GOVSECBID_11m', 'GOVSECBID_1m', 'GOVSECBID_1w',
       'GOVSECBID_1y', 'GOVSECBID_2m', 'GOVSECBID_3m', 'GOVSECBID_4m',
       'GOVSECBID_5m', 'GOVSECBID_6m', 'GOVSECBID_7m', 'GOVSECBID_8m',
       'GOVSECBID_9m', 'GBP_USD', 'LKR_LKR', 'USD_LKR', 'LKA36421B051',
       'LKA36421I031', 'LKB00523C152', 'LKB00524C150', 'LKB00524I157',
       'LKB00623E156', 'LKB01026H014', 'LKB01326B011', 'GLASN0000',
       'JETWN0000', 'JKHN0000', 'KHLN0000', 'LIOCN0000', 'LLUBN0000'],
      dtype='object')

In [61]:
##Obtains mean,sd of the data 
Combine1 = Combine.describe()

In [62]:
#calculating mean sd and drift for each column
mean = Combine1.loc['mean']
sd = Combine1.loc['std']
drift = mean - (sd * sd)/2

In [63]:
mean_df = pd.DataFrame(mean)
sd_df = pd.DataFrame(sd)
drift_df = pd.DataFrame(drift)

# Correlation Matrix

In [64]:
##Correlation matrix of the combined dataframe
corr = Combine.corr()
print(corr)

               GOVSECBID_10m  GOVSECBID_11m  GOVSECBID_1m  GOVSECBID_1w  \
GOVSECBID_10m       1.000000       0.988047      0.830957      0.820179   
GOVSECBID_11m       0.988047       1.000000      0.829924      0.818347   
GOVSECBID_1m        0.830957       0.829924      1.000000      0.981526   
GOVSECBID_1w        0.820179       0.818347      0.981526      1.000000   
GOVSECBID_1y        0.931614       0.955628      0.757175      0.744395   
GOVSECBID_2m        0.846267       0.851078      0.968116      0.948084   
GOVSECBID_3m        0.836876       0.832986      0.901266      0.873457   
GOVSECBID_4m        0.914496       0.900368      0.917276      0.894872   
GOVSECBID_5m        0.942067       0.933459      0.883879      0.863284   
GOVSECBID_6m        0.913518       0.909232      0.796469      0.782077   
GOVSECBID_7m        0.982312       0.961675      0.847310      0.833926   
GOVSECBID_8m        0.989951       0.971545      0.848129      0.834541   
GOVSECBID_9m        0.993

# Cholesky decomposition

Cholesky decomposition matrix:
Cholesky decomposition is an efficient method for inversion of symmetric positive-definite matrices. 
Any n*n symmetric positive definite matrix A  can be factored as A=LL^T 

In [65]:
C = np.linalg.cholesky(corr)

In [66]:
C_df =pd.DataFrame(C)

In [67]:
C_df.rename(columns = {0:'GOVSECBID_10m', 1:'GOVSECBID_11m',2:'GOVSECBID_1m',3:'GOVSECBID_1w', 4:'GOVSECBID_1y',5:'GOVSECBID_2m',6:'GOVSECBID_3m', 7:'GOVSECBID_4m',8:'GOVSECBID_5m',9:'GOVSECBID_6m', 10:'GOVSECBID_7m',11:'GOVSECBID_,8m',12:'GOVSECBID_9m', 13:'GBP_USD',14:'LKR_LKR',15:'USD_LKR', 16:'LKA36421B051',17:'LKA36421I031',18:'LKB00523C152', 19:'LKB00524C150',20:'LKB00524I157',21:'LKB00623E156', 22:'LKB01026H014',23:'LKB01326B011',24:'GLASN0000', 25:'JETWN0000',26:'JKHN0000',27:'KHLN0000', 28:'LIOCN0000',29:'LLUBN0000'}, inplace = True)

In [68]:
V = np.dot(C, C.T.conj()) # verify that C * C.H = A  
                          #(Where C is lower truangular matrix and H is transpose of C that is the lower traingular matrix)

In [69]:
V_df = pd.DataFrame(V)

In [70]:
V_df.rename(columns = {0:'GOVSECBID_10m', 1:'GOVSECBID_11m',2:'GOVSECBID_1m',3:'GOVSECBID_1w', 4:'GOVSECBID_1y',5:'GOVSECBID_2m',6:'GOVSECBID_3m', 7:'GOVSECBID_4m',8:'GOVSECBID_5m',9:'GOVSECBID_6m', 10:'GOVSECBID_7m',11:'GOVSECBID_,8m',12:'GOVSECBID_9m', 13:'GBP_USD',14:'LKR_LKR',15:'USD_LKR', 16:'LKA36421B051',17:'LKA36421I031',18:'LKB00523C152', 19:'LKB00524C150',20:'LKB00524I157',21:'LKB00623E156', 22:'LKB01026H014',23:'LKB01326B011',24:'GLASN0000', 25:'JETWN0000',26:'JKHN0000',27:'KHLN0000', 28:'LIOCN0000',29:'LLUBN0000'}, inplace = True)

In [71]:
##Generating random numbers between 0 and 1 for 1000*30 matrix 
random.seed(120)
std_random_simulations = generateRandoms(0,1,30000)
std_random_simulations

[0.5149379540077491,
 0.1886481713153758,
 0.9529195271072769,
 0.6352690810997838,
 0.5896618565242374,
 0.9404242818240754,
 0.09123847868402024,
 0.7928524256786783,
 0.22314811392015454,
 0.25117394866717546,
 0.32112440000938036,
 0.7394890871718477,
 0.9413394116528027,
 0.7062377607254028,
 0.3167918942808058,
 0.12845149846488024,
 0.2864339000852407,
 0.21754133098344886,
 0.5022412680752473,
 0.5088081116887921,
 0.1156031148493818,
 0.208781093048095,
 0.8786756411178476,
 0.7010222228582054,
 0.09491491320965939,
 0.8671078678224289,
 0.44899493365132404,
 0.3842184573031272,
 0.8051229931207687,
 0.6886186570173524,
 0.4363289020744926,
 0.3838244962667843,
 0.5441906747049473,
 0.8343204543762492,
 0.7081821142150656,
 0.08786560598080262,
 0.029798190242237288,
 0.3579457903944965,
 0.2891993427487628,
 0.7785937541239099,
 0.6077275888168364,
 0.13905950176129223,
 0.42131600459802,
 0.5766923004743484,
 0.48041543197742853,
 0.2583437587947791,
 0.6896640541094385,
 0.

In [72]:
#Creating a random matrix 1000*30 dimension
random_matrix = np.array(std_random_simulations).reshape(1000,30)
print(random_matrix)


[[0.51493795 0.18864817 0.95291953 ... 0.38421846 0.80512299 0.68861866]
 [0.4363289  0.3838245  0.54419067 ... 0.3300878  0.52720867 0.81919157]
 [0.15409386 0.84662065 0.92345651 ... 0.18130561 0.74182554 0.63149169]
 ...
 [0.15064486 0.55613517 0.59186516 ... 0.6418055  0.63033053 0.51468409]
 [0.79120958 0.3215904  0.78086132 ... 0.80669974 0.1240871  0.45307867]
 [0.46136215 0.25841408 0.4607931  ... 0.3027452  0.82338738 0.78500524]]


In [73]:
len(random_matrix)

1000

In [74]:
##Z matrix
Z_mat = stats.norm.ppf (random_matrix)
Z_mat

array([[ 0.03745265, -0.88288882,  1.67384561, ..., -0.29442009,
         0.86006355,  0.49193869],
       [-0.16028344, -0.29545151,  0.11099709, ..., -0.43967075,
         0.06825498,  0.91228851],
       [-1.01903212,  1.02204689,  1.42871217, ..., -0.91040072,
         0.64898368,  0.33580673],
       ...,
       [-1.03367161,  0.14117757,  0.2323455 , ...,  0.36328902,
         0.33272888,  0.03681586],
       [ 0.81062537, -0.46325611,  0.77510542, ...,  0.86579877,
        -1.15479544, -0.11788683],
       [-0.09700265, -0.64824244, -0.09843586, ..., -0.51652125,
         0.92835155,  0.78920959]])

In [75]:
C_T_df = C_df.transpose() ##Transposing rows to columns

In [76]:
##Multiplication of Z matrix and Cholesky Transposed matrix
simulated_Z_mat = np.mat(Z_mat) * np.mat(C_T_df)

In [77]:
simulated_Z_mat_df = pd.DataFrame(simulated_Z_mat)

In [78]:
x = Combine.columns


In [79]:
simulated_Z_mat_df.iloc[:,:]

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.037453,-0.099097,0.906345,0.949608,-0.198736,1.093885,0.185831,0.742223,0.433980,-0.200418,...,-0.461147,-0.444356,-0.043905,-0.111860,-0.728730,1.946919,0.974254,0.545955,0.554308,1.286231
1,-0.160283,-0.203913,-0.088827,0.098475,-0.082178,-0.413364,-0.866592,-0.478868,-0.553953,-0.372633,...,2.226676,2.235324,0.530847,0.625839,0.647489,0.290085,-0.245799,-0.083894,0.314705,0.548124
2,-1.019032,-0.849297,0.002786,-0.283948,-0.674932,-0.224324,-0.663547,-0.114314,-0.470674,-0.983485,...,-0.577271,-0.578306,1.124948,0.919410,-0.120563,0.211821,-0.318510,-1.138047,-0.416518,-0.634431
3,-0.208063,-0.343798,-0.770624,-0.771822,-0.215436,-0.824282,-1.344649,-0.674309,-0.680352,-0.536347,...,-0.400890,-0.384738,0.824847,0.510497,-0.839057,-0.992522,0.735787,0.519425,0.334923,0.111079
4,-0.762415,-0.613456,-0.782676,-0.827816,-0.869172,-0.917567,-1.032504,-0.669050,-0.713792,-1.153301,...,-0.436253,-0.445202,0.035247,0.044235,0.615975,-1.395188,-0.885386,0.326843,0.201302,-1.680045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.322966,0.169233,-0.189442,0.011574,0.355673,0.043309,0.010088,-0.511963,-0.256008,0.406460,...,-1.184427,-1.191444,-0.042714,-0.289673,0.543011,0.247266,-0.224841,-0.822070,-0.010107,-0.294480
996,0.953579,0.850247,0.816398,0.691571,0.947166,0.380667,0.395849,1.406082,1.832781,1.153844,...,-0.539063,-0.523226,1.307660,1.440325,-2.081664,-0.873154,0.730174,1.889381,0.201102,0.566782
997,-1.033672,-0.999552,-0.722222,-0.426988,-0.855034,-0.469869,-0.664334,-0.270386,-0.228799,-0.645180,...,2.621306,2.690206,1.956103,1.939933,-0.897219,0.237925,1.408305,1.214269,0.073130,-0.260882
998,0.810625,0.729522,1.075742,1.404662,0.634986,0.448353,0.923343,0.882113,0.387121,0.612478,...,0.138773,0.140559,1.807241,1.458690,-3.397191,0.465190,-0.658691,-0.528381,-2.150689,-1.837311


In [80]:
Simulated_return = {}
for i in range(len(Combine.columns)):
    s_return = list((drift_df.iloc[i,:].values + simulated_Z_mat_df.iloc[:,i]) * sd_df.iloc[i,:].values)
    Simulated_return[i] = s_return
    Simulated_return_df = pd.DataFrame(Simulated_return)

In [81]:
Simulated_return_df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.001266,-0.006606,0.041951,0.041978,-0.012712,0.053586,0.008327,0.035772,0.021750,-0.011773,...,-0.785413,-0.721644,-0.013198,-0.025349,-0.026610,0.057319,0.014241,0.013308,0.013319,0.030328
1,-0.009722,-0.012674,-0.004779,0.003843,-0.005789,-0.021147,-0.042437,-0.024146,-0.029341,-0.021253,...,1.620021,1.596410,0.089239,0.105715,0.023736,0.008514,-0.003607,-0.002080,0.007574,0.012934
2,-0.057443,-0.050034,-0.000477,-0.013292,-0.040997,-0.011774,-0.032643,-0.006258,-0.025034,-0.054882,...,-0.889336,-0.837517,0.195126,0.157873,-0.004362,0.006209,-0.004671,-0.027833,-0.009957,-0.014935
3,-0.012377,-0.020772,-0.036794,-0.035151,-0.013704,-0.041521,-0.065496,-0.033737,-0.035877,-0.030266,...,-0.731486,-0.670072,0.141639,0.085223,-0.030646,-0.029267,0.010753,0.012659,0.008059,0.002634
4,-0.043183,-0.036382,-0.037360,-0.037660,-0.052534,-0.046147,-0.050440,-0.033479,-0.037607,-0.064230,...,-0.763134,-0.722376,0.000909,0.002384,0.022583,-0.041128,-0.012964,0.007955,0.004855,-0.039576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.017132,0.008927,-0.009503,-0.000051,0.020218,0.001496,-0.000150,-0.025770,-0.013932,0.021637,...,-1.432703,-1.367912,-0.012986,-0.056941,0.019914,0.007253,-0.003301,-0.020114,-0.000213,-0.006923
996,0.052176,0.048351,0.037728,0.030417,0.055351,0.018223,0.018458,0.068348,0.094089,0.062781,...,-0.855143,-0.789870,0.227691,0.250422,-0.076104,-0.025751,0.010670,0.046128,0.004850,0.013374
997,-0.058257,-0.058733,-0.034521,-0.019701,-0.051694,-0.023949,-0.032681,-0.013916,-0.012525,-0.036257,...,1.973190,1.989904,0.343262,0.339186,-0.032774,0.006978,0.020591,0.029635,0.001782,-0.006132
998,0.044232,0.041362,0.049906,0.062367,0.036809,0.021579,0.043902,0.042637,0.019327,0.032978,...,-0.248522,-0.215665,0.316731,0.253685,-0.124230,0.013672,-0.009647,-0.012939,-0.051534,-0.043282


In [82]:
Simulated_return_df.rename(columns = {0:'GOVSECBID_10m', 1:'GOVSECBID_11m',2:'GOVSECBID_1m',3:'GOVSECBID_1w', 4:'GOVSECBID_1y',5:'GOVSECBID_2m',6:'GOVSECBID_3m', 7:'GOVSECBID_4m',8:'GOVSECBID_5m',9:'GOVSECBID_6m', 10:'GOVSECBID_7m',11:'GOVSECBID_,8m',12:'GOVSECBID_9m', 13:'GBP_USD',14:'LKR_LKR',15:'USD_LKR', 16:'LKA36421B051',17:'LKA36421I031',18:'LKB00523C152', 19:'LKB00524C150',20:'LKB00524I157',21:'LKB00623E156', 22:'LKB01026H014',23:'LKB01326B011',24:'GLASN0000', 25:'JETWN0000',26:'JKHN0000',27:'KHLN0000', 28:'LIOCN0000',29:'LLUBN0000'}, inplace = True)

In [83]:
###Simulated return for Instrument_type E
Simulated_return_df_E = Simulated_return_df.iloc[:,24:30]

In [84]:
df_E['value'][0]
df_E

,date,instrument,instrument_type,value,value_stressed,log_returns,returns_adjustment_factor,simulated_returns
0,2020-10-16,GLASN0000,E,7.1,7.1,-0.119347,9.171972e-07,-1.193458e-01
1,2020-10-15,GLASN0000,E,8.0,8.0,-0.095310,1.812227e-07,-9.531000e-02
2,2020-10-14,GLASN0000,E,8.8,8.8,0.228842,6.363912e-07,2.288422e-01
3,2020-10-13,GLASN0000,E,7.0,7.0,0.336472,2.516031e-07,3.364725e-01
4,2020-10-12,GLASN0000,E,5.0,5.0,0.174353,9.081317e-07,1.743543e-01
...,...,...,...,...,...,...,...,...
245,2019-11-08,LLUBN0000,E,65.1,65.1,0.000000,6.630170e-07,6.630170e-07
246,2019-11-07,LLUBN0000,E,65.1,65.1,-0.028768,9.101167e-07,-2.876716e-02
247,2019-11-06,LLUBN0000,E,67.0,67.0,-0.007435,6.358832e-07,-7.434343e-03
248,2019-11-05,LLUBN0000,E,67.5,67.5,-0.019076,1.129955e-07,-1.907603e-02


In [85]:
###Filtering according to Instrument
df_res_E = df_E.filter(['instrument', 'value']) ##To extract particular columns

In [86]:
df_res_E = df_res_E.set_index('instrument')

In [87]:
##Segregating simulations according to the key1 generated
current_value = (df_res_E.assign(labels = df_res_E.groupby(level = 0).cumcount())
            .groupby([df_res_E.index,'labels']).first()
            .unstack('labels')
            .sort_index(axis =1,level = 1)
            .droplevel(1,axis = 1))
current_value

,value,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
instrument,,,,,,,,,,,,,,,,,,,,,
GLASN0000,7.1,8.0,8.8,7.0,5.0,4.2,4.2,3.9,3.9,4.0,...,4.3,4.0,4.1,4.1,4.1,4.1,4.1,4.1,4.2,4.3
JETWN0000,9.1,9.0,9.2,9.0,9.0,8.9,9.0,9.0,8.5,8.7,...,12.5,12.0,12.0,10.8,10.8,10.8,10.8,10.8,10.8,10.8
JKHN0000,132.8,134.0,131.9,129.6,131.0,132.2,131.0,130.0,129.9,130.6,...,158.2,154.9,154.5,153.5,153.5,153.5,153.5,154.1,154.0,155.0
KHLN0000,8.3,8.0,8.1,8.0,8.3,8.1,8.2,7.8,7.7,7.9,...,10.6,10.4,10.4,10.2,10.2,10.2,10.2,10.2,10.3,10.2
LIOCN0000,24.7,24.7,24.0,23.6,24.4,24.5,23.7,22.8,22.1,22.6,...,17.0,17.0,17.0,16.6,16.6,16.6,16.6,16.7,16.8,17.0
LLUBN0000,89.9,88.5,85.5,84.0,84.8,84.1,82.6,80.5,79.5,80.1,...,69.1,68.5,67.6,65.1,65.1,65.1,65.1,67.0,67.5,68.8


In [88]:
current_value_E = current_value.iloc[:,0]

In [89]:
Simulated_return_df_E

,GLASN0000,JETWN0000,JKHN0000,KHLN0000,LIOCN0000,LLUBN0000
0,-0.026610,0.057319,0.014241,0.013308,0.013319,0.030328
1,0.023736,0.008514,-0.003607,-0.002080,0.007574,0.012934
2,-0.004362,0.006209,-0.004671,-0.027833,-0.009957,-0.014935
3,-0.030646,-0.029267,0.010753,0.012659,0.008059,0.002634
4,0.022583,-0.041128,-0.012964,0.007955,0.004855,-0.039576
...,...,...,...,...,...,...
995,0.019914,0.007253,-0.003301,-0.020114,-0.000213,-0.006923
996,-0.076104,-0.025751,0.010670,0.046128,0.004850,0.013374
997,-0.032774,0.006978,0.020591,0.029635,0.001782,-0.006132
998,-0.124230,0.013672,-0.009647,-0.012939,-0.051534,-0.043282


In [90]:
EXP = np.exp(Simulated_return_df_E)

In [91]:
Simulated_rates_E = EXP*current_value_E

In [92]:
##Current value of Simulated_rates_E

In [93]:
Simulated_return_df.columns

Index(['GOVSECBID_10m', 'GOVSECBID_11m', 'GOVSECBID_1m', 'GOVSECBID_1w',
       'GOVSECBID_1y', 'GOVSECBID_2m', 'GOVSECBID_3m', 'GOVSECBID_4m',
       'GOVSECBID_5m', 'GOVSECBID_6m', 'GOVSECBID_7m', 'GOVSECBID_,8m',
       'GOVSECBID_9m', 'GBP_USD', 'LKR_LKR', 'USD_LKR', 'LKA36421B051',
       'LKA36421I031', 'LKB00523C152', 'LKB00524C150', 'LKB00524I157',
       'LKB00623E156', 'LKB01026H014', 'LKB01326B011', 'GLASN0000',
       'JETWN0000', 'JKHN0000', 'KHLN0000', 'LIOCN0000', 'LLUBN0000'],
      dtype='object')

In [94]:
###Simulated return for r|ate_exchange dataset
Simulated_return_df_Exchange = Simulated_return_df.iloc[:,13:16]

In [95]:
###Filtering according to Instrument
df2_res = df2.filter(['key2', 'rate']) ##To extract particular columns

In [96]:
df2_res = df2_res.set_index('key2')

In [97]:
##Segregating values according to the key2 generated
current_value1 = (df2_res.assign(labels = df2_res.groupby(level = 0).cumcount())
            .groupby([df2_res.index,'labels']).first()
            .unstack('labels')
            .sort_index(axis =1,level = 1)
            .droplevel(1,axis = 1))

In [98]:
current_value_Exchange = current_value1.iloc[:,0]

In [99]:
np.exp(Simulated_return_df_Exchange)

,GBP_USD,LKR_LKR,USD_LKR
0,1.026058,1.0,0.952775
1,1.001476,1.0,0.978357
2,1.063816,1.0,0.944605
3,1.008414,1.0,1.024950
4,0.978590,1.0,0.987273
...,...,...,...
995,1.044730,1.0,1.000223
996,1.039198,1.0,1.015953
997,1.023293,1.0,0.971606
998,1.007694,1.0,0.946598


In [100]:
Simulated_rates_Exchange = np.exp(Simulated_return_df_Exchange)*current_value_Exchange

# Transaction Equity Dataset

In [101]:
##Importing dataset rates_interest
df3= getData(path3)

In [102]:
##Sorting data by'instrument' column
df3= df3.sort_values(by=['instrument']).reset_index(drop=True)

In [103]:
#Creating currency LKR_LKR 
df3["key3"] = df3["currency"] + '_' + df3["currency_base"]

# Crossrate

In [104]:
##We have fetched 2nd column of the dataframe Simulated_rates_Exchange because in our transaction equity file, the currency is LKR_LKR

crossrate = Simulated_rates_Exchange.iloc[:,1]

In [105]:
##Fetching second value of the current_value_Exchange series 
current_value_Exchange_2nd = pd.DataFrame(current_value_Exchange).iloc[1]
current_value_Exchange_2nd

rate    1.0
Name: LKR_LKR, dtype: float64

In [106]:
##Appending original current rate_exchange
crossrate = pd.concat([(current_value_Exchange_2nd),crossrate.loc[:]]).reset_index(drop=True)

In [107]:
###Adding current value of original equity instrumnet E data row wiseto the Simulated_rates_E
##In this code we are converting the current value series to dataframe and then transposing it and appending to the simulated rates df
Equity_Rate = pd.concat([pd.DataFrame(current_value_E).transpose(),Simulated_rates_E.loc[:]]).reset_index(drop=True)

# GLASN0000

In [108]:
Instrument1 = Equity_Rate.iloc[:,0]

In [109]:
unit_count1 = df3['unit_count'][0]

In [110]:
unit_counts = [unit_count1 for i in range(1001)]

In [111]:
unit_counts_df = pd.DataFrame(unit_counts)

In [112]:
##Renamimg the indexed column
unit_counts_df = unit_counts_df.rename(columns={unit_counts_df.columns[0]: 'unit_counts'})

In [113]:
##Appending original current rate_exchange
Result1 = pd.concat([Instrument1, crossrate , unit_counts_df], axis=1)

In [114]:
Result1['MTM'] = Result1['GLASN0000']*Result1['LKR_LKR']*Result1['unit_counts']

In [115]:
Result1['MTM'][0] ##Fetching the current value of instrument'GLASN0000' from equity dataset

4454547.1

In [116]:
##Change in MTM
#MTM-Current_MTM
Result1['Change_in_MTM'] = Result1['MTM']-Result1['MTM'][0]

In [117]:
##Replicationg instrument name that is GLASN0000 1001 number of times 
Instrument_name="GLASN0000" 
lis = []
for i in range(1001):
    a = Instrument_name
    print(Instrument_name) 
    lis.append(a)

GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000
GLASN0000


In [118]:
Instrument_name= pd.DataFrame(lis) ##Converting list to DataFrame

In [119]:
##Inserting the instrument name column in the Result1 dataframe
idx = 0
Result1.insert(loc=idx, column='Instrument_name', value=lis)
Result1

,Instrument_name,GLASN0000,LKR_LKR,unit_counts,MTM,Change_in_MTM
0,GLASN0000,7.100000,1.0,627401,4.454547e+06,0.000000
1,GLASN0000,6.913560,1.0,627401,4.337574e+06,-116973.133678
2,GLASN0000,7.270541,1.0,627401,4.561544e+06,106997.211288
3,GLASN0000,7.069100,1.0,627401,4.435160e+06,-19387.178684
4,GLASN0000,6.885712,1.0,627401,4.320103e+06,-134444.470358
...,...,...,...,...,...,...
996,GLASN0000,7.242805,1.0,627401,4.544144e+06,89596.559290
997,GLASN0000,6.579709,1.0,627401,4.128115e+06,-326432.319956
998,GLASN0000,6.871077,1.0,627401,4.310920e+06,-143626.992389
999,GLASN0000,6.270555,1.0,627401,3.934154e+06,-520392.884485


In [120]:
Result1.rename(columns = {'GLASN0000':'Rates'}, inplace = True)
Result1

,Instrument_name,Rates,LKR_LKR,unit_counts,MTM,Change_in_MTM
0,GLASN0000,7.100000,1.0,627401,4.454547e+06,0.000000
1,GLASN0000,6.913560,1.0,627401,4.337574e+06,-116973.133678
2,GLASN0000,7.270541,1.0,627401,4.561544e+06,106997.211288
3,GLASN0000,7.069100,1.0,627401,4.435160e+06,-19387.178684
4,GLASN0000,6.885712,1.0,627401,4.320103e+06,-134444.470358
...,...,...,...,...,...,...
996,GLASN0000,7.242805,1.0,627401,4.544144e+06,89596.559290
997,GLASN0000,6.579709,1.0,627401,4.128115e+06,-326432.319956
998,GLASN0000,6.871077,1.0,627401,4.310920e+06,-143626.992389
999,GLASN0000,6.270555,1.0,627401,3.934154e+06,-520392.884485


# JETWN0000

In [121]:
#JETWN0000
Instrument2 = Equity_Rate.iloc[:,1]
Result2 = pd.concat([Instrument2, crossrate , unit_counts_df], axis=1)
Result2['MTM'] = Result2['JETWN0000']*Result2['LKR_LKR']*Result2['unit_counts']
Result2['Change_in_MTM'] = Result2['MTM']-Result2['MTM'][0]

In [122]:
##Replicationg instrument name that is JETWN0000 1001 number of times 
Instrument_name="JETWN0000" 
lis = []
for i in range(1001):
    a = Instrument_name
    print(Instrument_name) 
    lis.append(a)

JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000
JETWN0000


In [123]:
Instrument_name= pd.DataFrame(lis) ##Converting list to DataFrame

In [124]:
##Inserting the instrument name column in the Result2 dataframe
idx = 0
Result2.insert(loc=idx, column='Instrument_name', value=lis)
Result2

,Instrument_name,JETWN0000,LKR_LKR,unit_counts,MTM,Change_in_MTM
0,JETWN0000,9.100000,1.0,627401,5.709349e+06,0.000000
1,JETWN0000,9.636838,1.0,627401,6.046161e+06,336811.701785
2,JETWN0000,9.177809,1.0,627401,5.758167e+06,48817.414228
3,JETWN0000,9.156676,1.0,627401,5.744907e+06,35557.737831
4,JETWN0000,8.837531,1.0,627401,5.544675e+06,-164673.891239
...,...,...,...,...,...,...
996,JETWN0000,9.166241,1.0,627401,5.750909e+06,41560.011497
997,JETWN0000,8.868659,1.0,627401,5.564204e+06,-145145.102722
998,JETWN0000,9.163719,1.0,627401,5.749325e+06,39976.383330
999,JETWN0000,9.225270,1.0,627401,5.787947e+06,78597.657829


In [125]:
Result2.rename(columns = {'JETWN0000':'Rates'}, inplace = True)
Result2

,Instrument_name,Rates,LKR_LKR,unit_counts,MTM,Change_in_MTM
0,JETWN0000,9.100000,1.0,627401,5.709349e+06,0.000000
1,JETWN0000,9.636838,1.0,627401,6.046161e+06,336811.701785
2,JETWN0000,9.177809,1.0,627401,5.758167e+06,48817.414228
3,JETWN0000,9.156676,1.0,627401,5.744907e+06,35557.737831
4,JETWN0000,8.837531,1.0,627401,5.544675e+06,-164673.891239
...,...,...,...,...,...,...
996,JETWN0000,9.166241,1.0,627401,5.750909e+06,41560.011497
997,JETWN0000,8.868659,1.0,627401,5.564204e+06,-145145.102722
998,JETWN0000,9.163719,1.0,627401,5.749325e+06,39976.383330
999,JETWN0000,9.225270,1.0,627401,5.787947e+06,78597.657829


# JKHN0000


In [126]:
Instrument3 = Equity_Rate.iloc[:,2]
Result3 = pd.concat([Instrument3, crossrate , unit_counts_df], axis=1)
Result3['MTM'] = Result3['JKHN0000']*Result3['LKR_LKR']*Result3['unit_counts']
Result3['Change_in_MTM'] = Result3['MTM']-Result3['MTM'][0]

In [127]:
##Replicationg instrument name that is JETWN0000 1001 number of times 
Instrument_name="JKHN0000" 
lis = []
for i in range(1001):
    a = Instrument_name
    print(Instrument_name) 
    lis.append(a)

JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
JKHN0000
J

In [128]:
Instrument_name= pd.DataFrame(lis) ##Converting list to DataFrame

In [129]:
##Inserting the instrument name column in the Result2 dataframe
idx = 0
Result3.insert(loc=idx, column='Instrument_name', value=lis)
Result3.rename(columns = {'JKHN0000':'Rates'}, inplace = True)
Result1

,Instrument_name,Rates,LKR_LKR,unit_counts,MTM,Change_in_MTM
0,GLASN0000,7.100000,1.0,627401,4.454547e+06,0.000000
1,GLASN0000,6.913560,1.0,627401,4.337574e+06,-116973.133678
2,GLASN0000,7.270541,1.0,627401,4.561544e+06,106997.211288
3,GLASN0000,7.069100,1.0,627401,4.435160e+06,-19387.178684
4,GLASN0000,6.885712,1.0,627401,4.320103e+06,-134444.470358
...,...,...,...,...,...,...
996,GLASN0000,7.242805,1.0,627401,4.544144e+06,89596.559290
997,GLASN0000,6.579709,1.0,627401,4.128115e+06,-326432.319956
998,GLASN0000,6.871077,1.0,627401,4.310920e+06,-143626.992389
999,GLASN0000,6.270555,1.0,627401,3.934154e+06,-520392.884485


# KHLN0000

In [130]:
Instrument4 = Equity_Rate.iloc[:,3]
Result4 = pd.concat([Instrument4, crossrate , unit_counts_df], axis=1)
Result4['MTM'] = Result4['KHLN0000']*Result4['LKR_LKR']*Result4['unit_counts']
Result4['Change_in_MTM'] = Result4['MTM']-Result4['MTM'][0]


In [131]:
##Replicationg instrument name that is JETWN0000 1001 number of times 
Instrument_name="KHLN0000" 
lis = []
for i in range(1001):
    a = Instrument_name
    print(Instrument_name) 
    lis.append(a)

KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
KHLN0000
K

In [132]:
Instrument_name= pd.DataFrame(lis) ##Converting list to DataFrame

In [133]:
##Inserting the instrument name column in the Result2 dataframe
idx = 0
Result4.insert(loc=idx, column='Instrument_name', value=lis)
Result4.rename(columns = {'KHLN0000':'Rates'}, inplace = True)
Result4

,Instrument_name,Rates,LKR_LKR,unit_counts,MTM,Change_in_MTM
0,KHLN0000,8.300000,1.0,627401,5.207428e+06,0.000000
1,KHLN0000,8.411190,1.0,627401,5.277189e+06,69760.386492
2,KHLN0000,8.282756,1.0,627401,5.196609e+06,-10819.383811
3,KHLN0000,8.072172,1.0,627401,5.064488e+06,-142939.870034
4,KHLN0000,8.405741,1.0,627401,5.273770e+06,66341.526332
...,...,...,...,...,...,...
996,KHLN0000,8.134725,1.0,627401,5.103735e+06,-103693.018147
997,KHLN0000,8.691826,1.0,627401,5.453259e+06,245830.646251
998,KHLN0000,8.549647,1.0,627401,5.364056e+06,156628.068854
999,KHLN0000,8.193301,1.0,627401,5.140488e+06,-66940.657047


# LIOCN0000

In [134]:
Instrument5 = Equity_Rate.iloc[:,4]
Result5 = pd.concat([Instrument5, crossrate , unit_counts_df], axis=1)
Result5['MTM'] = Result5['LIOCN0000']*Result5['LKR_LKR']*Result5['unit_counts']
Result5['Change_in_MTM'] = Result5['MTM']-Result5['MTM'][0]


In [135]:
##Replicationg instrument name that is JETWN0000 1001 number of times 
Instrument_name="LIOCN0000" 
lis = []
for i in range(1001):
    a = Instrument_name
    print(Instrument_name) 
    lis.append(a)

LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000
LIOCN0000


In [136]:
Instrument_name= pd.DataFrame(lis) ##Converting list to DataFrame

In [137]:
##Inserting the instrument name column in the Result2 dataframe
idx = 0
Result5.insert(loc=idx, column='Instrument_name', value=lis)
Result5.rename(columns = {'LIOCN0000':'Rates'}, inplace = True)
Result5

,Instrument_name,Rates,LKR_LKR,unit_counts,MTM,Change_in_MTM
0,LIOCN0000,24.700000,1.0,627401,1.549680e+07,0.000000
1,LIOCN0000,25.031169,1.0,627401,1.570458e+07,207773.599714
2,LIOCN0000,24.887789,1.0,627401,1.561462e+07,117818.116213
3,LIOCN0000,24.455279,1.0,627401,1.534327e+07,-153539.459383
4,LIOCN0000,24.899856,1.0,627401,1.562219e+07,125388.600009
...,...,...,...,...,...,...
996,LIOCN0000,24.694730,1.0,627401,1.549350e+07,-3304.696070
997,LIOCN0000,24.820096,1.0,627401,1.557215e+07,75343.503557
998,LIOCN0000,24.744061,1.0,627401,1.552445e+07,27641.259059
999,LIOCN0000,23.459351,1.0,627401,1.471843e+07,-778377.253385


# LLUBN0000


In [138]:
Instrument6 = Equity_Rate.iloc[:,5]
Result6 = pd.concat([Instrument6, crossrate , unit_counts_df], axis=1)
Result6['MTM'] = Result6['LLUBN0000']*Result6['LKR_LKR']*Result6['unit_counts']
Result6['Change_in_MTM'] = Result6['MTM']-Result6['MTM'][0]


In [139]:
##Replicationg instrument name that is JETWN0000 1001 number of times 
Instrument_name="LLUBN0000" 
lis = []
for i in range(1001):
    a = Instrument_name
    print(Instrument_name) 
    lis.append(a)

LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000
LLUBN0000


In [140]:
Instrument_name= pd.DataFrame(lis) ##Converting list to DataFrame

In [141]:
##Inserting the instrument name column in the Result2 dataframe
idx = 0
Result6.insert(loc=idx, column='Instrument_name', value=lis)
Result6.rename(columns = {'LLUBN0000':'Rates'}, inplace = True)
Result6

,Instrument_name,Rates,LKR_LKR,unit_counts,MTM,Change_in_MTM
0,LLUBN0000,89.900000,1.0,627401,5.640335e+07,0.000000e+00
1,LLUBN0000,92.668296,1.0,627401,5.814017e+07,1.736825e+06
2,LLUBN0000,91.070308,1.0,627401,5.713760e+07,7.342491e+05
3,LLUBN0000,88.567339,1.0,627401,5.556723e+07,-8.361185e+05
4,LLUBN0000,90.137132,1.0,627401,5.655212e+07,1.487732e+05
...,...,...,...,...,...,...
996,LLUBN0000,89.279742,1.0,627401,5.601420e+07,-3.891456e+05
997,LLUBN0000,91.110361,1.0,627401,5.716271e+07,7.593643e+05
998,LLUBN0000,89.350459,1.0,627401,5.605856e+07,-3.447915e+05
999,LLUBN0000,86.091912,1.0,627401,5.401418e+07,-2.389172e+06


In [142]:
pd.set_option('display.max_rows', None) ##To read all the rows of the data
portfolio = pd.concat([Result1,Result2,Result3,Result4,Result5,Result6],axis=0)
portfolio.rename(columns = {'LKR_LKR':'Cross_rates'}, inplace = True)
portfolio

,Instrument_name,Rates,Cross_rates,unit_counts,MTM,Change_in_MTM
0,GLASN0000,7.100000,1.000000,627401,4.454547e+06,0.000000e+00
1,GLASN0000,6.913560,1.000000,627401,4.337574e+06,-1.169731e+05
2,GLASN0000,7.270541,1.000000,627401,4.561544e+06,1.069972e+05
3,GLASN0000,7.069100,1.000000,627401,4.435160e+06,-1.938718e+04
4,GLASN0000,6.885712,1.000000,627401,4.320103e+06,-1.344445e+05
5,GLASN0000,7.262164,1.000000,627401,4.556288e+06,1.017406e+05
6,GLASN0000,6.768247,1.000000,627401,4.246405e+06,-2.081419e+05
7,GLASN0000,6.988279,1.000000,627401,4.384454e+06,-7.009280e+04
8,GLASN0000,7.574231,1.000000,627401,4.752080e+06,2.975325e+05
9,GLASN0000,6.947596,1.000000,627401,4.358929e+06,-9.561800e+04


In [143]:
##Sum of change in MTM index wise
lis1 = []
for i in range(1001):
    Total_change_in_MTM = portfolio['Change_in_MTM'][i].sum()
    b= Total_change_in_MTM
    print(Total_change_in_MTM)
    lis1.append(b)

0.0
3429227.4901914345
697054.2666649222
-1504698.7917074272
942101.952118909
-3273129.7330596093
-5512008.305662806
-205266.53367535677
4887282.452021874
-324130.0238908641
-2385680.9646327198
-663936.4445034489
959794.481538754
-434072.26511786226
193731.3254495836
1164431.6405709973
859091.1549680792
384981.18340945896
1200933.5382355629
2917215.81596472
13864.620165572502
415498.5992884319
505043.51745236386
2962691.862858368
4263588.100520266
3923990.942464823
-1076883.5222444143
130166.51780133322
633395.7910087882
561869.5288425311
2842812.615064284
-57382.22665529046
-236643.54637792613
-2872985.721283328
-2325877.004367386
2782660.6200139383
-1474061.5774336047
-5521945.524185204
-3480279.1944899233
-6751851.937489712
-2041061.567746453
-2865011.419211354
2531187.822034061
2148337.152449955
5614379.508050544
-2054202.5194931747
-170154.94739955477
-20540.284671949223
-2497743.975835164
-2596768.0067949984
-1847092.2470195182
-5361514.653722385
-2374814.1138396645
-668934.52577

In [144]:
##Converting to dataframe
Total_change_in_MTM_df = pd.DataFrame(lis1)

In [145]:
#Dropping 0th index
Total_change_in_MTM_df = Total_change_in_MTM_df.iloc[1: , :]
Total_change_in_MTM_df


,0
1,3.429227e+06
2,6.970543e+05
3,-1.504699e+06
4,9.421020e+05
5,-3.273130e+06
6,-5.512008e+06
7,-2.052665e+05
8,4.887282e+06
9,-3.241300e+05
10,-2.385681e+06


In [146]:
##Renamimg the indexed column
Total_change_in_MTM_df = Total_change_in_MTM_df.rename(columns={Total_change_in_MTM_df.columns[0]: 'Total_change_in_MTM'})

In [147]:
#Sorting in descendingo
Total_change_in_MTM_df = Total_change_in_MTM_df.sort_values(by = 'Total_change_in_MTM', ascending=False)

# VaR

In [148]:
VaR_Exact=-Total_change_in_MTM_df.quantile(1-0.99, axis = 0)

# df =0 Nearby VaR

In [149]:
##Importing dataset rates_instrument
param = getData(path4)
param

,no_of_simulations,alpha,random_seed,decay_factor,holding_period
0,1000,0.99,120,0,1


In [150]:
##Extracting parameters
decay_factor = param['decay_factor']
holding_period = param['holding_period']
confidence_level = param['alpha']

In [151]:
##Using Math.floor function to round off to nearest integer
nearby = math.floor(confidence_level * len(Total_change_in_MTM_df))
nearby_nxt = nearby + 1
nearby_nxt

991

In [152]:
Total_change_in_MTM_df.iloc[nearby]

Total_change_in_MTM   -6.392768e+06
Name: 629, dtype: float64

In [153]:
# Transform as numeric
TCM = Total_change_in_MTM_df.astype(int)
VE = VaR_Exact.astype(int)

In [154]:
diff1 = (VE + TCM.iloc[nearby]).astype(int)

In [155]:
diff2 = (- TCM.iloc[nearby_nxt] - VE).astype(int)

In [156]:
##To calculate VaR nearby
if (diff1 == diff2).bool():
    VaR_nearby = - TCM.iloc[nearby + 1]

elif (diff1 < diff2).bool():
    VaR_nearby = - TCM.iloc[nearby]
    
else:
    VaR_nearby = - TCM.iloc[nearby + 1]

In [157]:
VaR_nearby ###here name is not changed instead it is showing the previous variables

Total_change_in_MTM    6392767
Name: 629, dtype: int64